# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Цель работы**

Изучить модель дерева решений, случайного леса и логистической регресии. Выбрать лучшую модель с самым высоким качеством. Оценить адекватность моделей.

**Ход работы**

Данные мы получим из файла  `/datasets/users_behavior.csv`. Поскольку с данными раньше уже проводилась работа, то предобработку данных можно не делать. После изучения данных можно сразу приступить к работе с моделями.

Работа пройдёт в пять этапов:
 1. Обзор данных.
 2. Разбить данные на выборки.
 3. Исследование моделей.
 4. Проверка моделей на тестовой выборке.
 5. Проверка моделей на адекватность.

## Откройте и изучите файл

In [1]:
#импортируем необходимые библиотеки
#подключим pandas
import pandas as pd
#подключим классификацю деревом решений
from sklearn.tree import DecisionTreeClassifier
#вычисление доли правильных ответов
from sklearn.metrics import accuracy_score
#разделение на выборки
from sklearn.model_selection import train_test_split
#инициализация модели случайного леса
from sklearn.ensemble import RandomForestClassifier
#инициализация модели логистической регрессии
from sklearn.linear_model import LogisticRegression
#вычесление точности 
from sklearn.metrics import precision_score
#вычесление полноты
from sklearn.metrics import recall_score
#вычесление f-меры
from sklearn.metrics import f1_score

In [2]:
#прочитаем файл csv
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
#выведем первые 10 строк таблицы
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
#выведем основную информацию о таблице методом info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#оценим данные в таблице методом describe
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Выводы**

В каждой строке таблицы - информация о поведении одного пользователя за месяц. Колонки описывают следуюущю информацию: количество звонков, суммарная длительность звонков в минутах, количество sms-сообщений, израсходованный интернет-трафик в Мб и каким тарифом пользовался в течение месяца.

Составим таблицы признаков и разделим данные на выборки.

## Разбейте данные на выборки

In [6]:
#составим таблицы признаков и целевого признака
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

#разобьём данные на три выборки: обучающую, валидационную и тестовую
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, train_size=0.6, random_state=46)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=46)

print('Размер обучающей выборки:', features_train.shape[0],'\n'
      'Размер валидационной выборки:', features_valid.shape[0],'\n'
      'Размер обучающей выборки:', features_test.shape[0])

Размер обучающей выборки: 1928 
Размер валидационной выборки: 643 
Размер обучающей выборки: 643


**Вывод**

За целевой признак берём столбец is_ultra.

Получили три набора данных с размерами: обучающая 60%, валидационная 20%, тестовая 20%. Теперь можем приступить к созданию моделей. Проверим три модели: дерево решений, случайный лес и логистическая регрессия.

## Исследуйте модели

Для начала мы обучим модели на обучающей выборке и посчитаем качество на валидационной выборке. Тестовую мы оставим на потом.

### Модель дерева решений

In [7]:
#Дерево решений
best_result1 = 0
best_depth = None
best_crit = ''
best_split = ''
best_model_tree = None
for depth in range(1, 15):
    for crit in {'gini', 'entropy'}:      #проверим модель на каждом критерии
        for split in {'best', 'random'}:
            #создадим модель дерева решений
            model1 = DecisionTreeClassifier(random_state=42, max_depth=depth, criterion=crit, splitter=split)               
            model1.fit(features_train, target_train)            #обучим модель на тренировочной выборке
            predictions = model1.predict(features_valid)        #получим предсказания модели
            result1 = accuracy_score(target_valid, predictions) #посчитаем качество модели на тестовой выборке
            if result1 > best_result1:
                best_result1 = result1   #получим результат с лучшей правильностью
                best_depth = depth
                best_crit = crit
                best_split = split
                best_model_tree = model1
    
print('Дерево решений:', best_result1, ', ' 'лучшая глубина:', best_depth, ', ' 'лучший критерий:', best_crit, ', ' 'лучший разветвитель:', best_split)

Дерево решений: 0.7853810264385692 , лучшая глубина: 6 , лучший критерий: entropy , лучший разветвитель: best


### Модель случайного леса

In [8]:
#Случайный лес
best_result2 = 0
best_est = None
best_criter = ''
best_model_forest = None
for est in range(1, 20):
    for criter in {'gini', 'entropy'}:   #проверим модель на каждом критерии
        model2 = RandomForestClassifier(random_state=42, n_estimators=est, criterion=criter) # создадим модель с заданным количеством деревьев
        result2 = (model2
                    .fit(features_train, target_train)
                    .score(features_valid, target_valid))
        if result2 > best_result2:
            best_result2 = result2
            best_est = est
            best_criter = criter
            best_model_forest = model2
            
print('Случайный лес:', best_result2, ', ' 'лучшее дерево:', best_est, ', ' 'лучший критерий:', best_criter)

Случайный лес: 0.7776049766718507 , лучшее дерево: 11 , лучший критерий: gini


### Модель логистической регрессии

In [9]:
#Логистическая регрессия
best_result3 = 0
best_it = None
best_sol = ''
best_model_reg = None
for it in range(100, 200):
    for sol in {'lbfgs', 'liblinear'}:   #проверим алгоритм
        model3 = LogisticRegression(random_state=42, solver=sol, max_iter=it) #создадим модель логистической регрессии
        result3 = (model3
                    .fit(features_train, target_train)
                    .score(features_valid, target_valid))
        if result3 > best_result3:
            best_result3 = result3
            best_it = it
            best_sol = sol
            best_model_reg = model3
            
print('Логистическая регрессия:', best_result3, ', ' 'лучшая итерация:', best_it, ', ' 'лучший алгоритм:', best_sol)

Логистическая регрессия: 0.7356143079315708 , лучшая итерация: 100 , лучший алгоритм: lbfgs


**Вывод**

По полученным результатам можно сделать следующие выводы:

* Дерево решений получило самую большую правильность при глубине в 6 с критерием entropy
* Случайный лес получил самую большую правильность на 11 дереве с критерием gini
* Логистическая регрессия определила самую большую правильность на 100 итерации с алогритмом lbfgs

Пока лучший ответ даёт дерево решений. Теперь проверим модели на тестовой выборке. 

## Проверьте модель на тестовой выборке

In [10]:
#объединим обучащую и валидационную выборку
features_full_train = pd.concat([features_train, features_valid])
target_full_train = pd.concat([target_train, target_valid])

#проверим дерево решений на тестовой выборке
best_tree_test = (best_model_tree
                  .fit(features_full_train, target_full_train)
                  .score(features_test, target_test))
print('Accuracy лучшей модели дерева на тестовой выборке:', best_tree_test)

Accuracy лучшей модели дерева на тестовой выборке: 0.7931570762052877


In [11]:
#проверим случайный лес на тестовой выборке
best_forest_test = (best_model_forest
                    .fit(features_full_train, target_full_train)
                    .score(features_test, target_test))
print('Accuracy лучшей модели случайного леса на тестовой выборке:', best_forest_test)

Accuracy лучшей модели случайного леса на тестовой выборке: 0.7869362363919129


In [12]:
#проверим логистическую регрессию на тестовой выборке
best_reg_test = (best_model_reg
                    .fit(features_full_train, target_full_train)
                    .score(features_test, target_test))
print('Accuracy лучшей модели логистической регрессии на тестовой выборке:', best_reg_test)

Accuracy лучшей модели логистической регрессии на тестовой выборке: 0.7231726283048211


**Выводы**

Можно сказать, что не все модели показали хороший результат. Правильность модели дерева решений выросла на 0.8 пункта, а случайный лес вырос на 1 пункт. Логистическая регрессия показала меньший результат на 1.3 пункта. Лучше всех себя показало дерево решений и случайный лес. 


## (бонус) Проверьте модели на адекватность

Оценить адекватность в задании можно по следующим метрикам:

1. Отношение числа правильных ответов к размеру тестовой выборки называется accuracy (англ. «правильность»)
2. Точность (англ. precision) показывает, какая доля тарифов, отмеченных моделью как 'Ультра', действительно являются ультра (ответ 1).
3. Полнота (англ. recall) выявляет, какую часть 'Ультра' объектов выделила модель.

Поскольку значение accuracy мы уже получили, то посчитаем только precision и recall для всех моделей.

In [13]:
#посчитаем точность и полноту для дерева решений
predictions_tree = best_model_tree.predict(features_test)
precision = precision_score(target_test, predictions_tree)
recall = recall_score(target_test, predictions_tree)

print('Precision =', precision, 'Recall =', recall)

Precision = 0.7241379310344828 Recall = 0.4540540540540541


In [14]:
#посчитаем точность и полноту для случайного леса
predictions_forest = best_model_forest.predict(features_test)
precision = precision_score(target_test, predictions_forest)
recall = recall_score(target_test, predictions_forest)

print('Precision =', precision, 'Recall =', recall)

Precision = 0.6304347826086957 Recall = 0.6270270270270271


In [15]:
#посчитаем точность и полноту для логистической регрессии
predictions_reg = best_model_reg.predict(features_test)
precision = precision_score(target_test, predictions_reg)
recall = recall_score(target_test, predictions_reg)

print('Precision =', precision, 'Recall =', recall)

Precision = 0.7058823529411765 Recall = 0.06486486486486487


F-мера

Понятно что чем выше точность и полнота, тем лучше. F-мера представляет собой гармоническое среднее между точностью и полнотой. Она стремится к нулю, если точность или полнота стремится к нулю. 

In [16]:
#f-мера для дерева решений
f_tree = f1_score(target_test, predictions_tree)
print('F-мера =', f_tree)

F-мера = 0.5581395348837209


In [17]:
#f-мера для случайного леса
f_forest = f1_score(target_test, predictions_forest)
print('F-мера =', f_forest)

F-мера = 0.6287262872628727


In [18]:
#f-мера для логистической регрессии
f_reg = f1_score(target_test, predictions_reg)
print('F-мера =', f_reg)

F-мера = 0.11881188118811882


**Вывод**

F-мера представляет из себя единую оценку адекватности модели, которая складывается из точности и полноты. По полученным значениям можно сказать, что модель случайного леса самая адекватная. Дерево немного отстаёт от леса, а вот регрессия явный аутсайдер.

In [19]:
#создадим "глупую" модель и заполним её единицами
dummy_prediction = [1] * len(target_test)

In [20]:
print('Accuracy "глупой" модели дерева решений:', best_model_tree.score(features_test, dummy_prediction))
print('Accuracy лучшей модели дерева решений:', best_model_tree.score(features_test, target_test))

Accuracy "глупой" модели дерева решений: 0.18040435458786935
Accuracy лучшей модели дерева решений: 0.7931570762052877


In [21]:
print('Accuracy "глупой" модели случайного леса:', best_model_forest.score(features_test, dummy_prediction))
print('Accuracy лучшей модели случайного леса:', best_model_forest.score(features_test, target_test))

Accuracy "глупой" модели случайного леса: 0.28615863141524106
Accuracy лучшей модели случайного леса: 0.7869362363919129


In [22]:
print('Accuracy "глупой" модели логистической регрессии:', best_model_reg.score(features_test, dummy_prediction))
print('Accuracy лучшей модели логистической регресии:', best_model_reg.score(features_test, target_test))

Accuracy "глупой" модели логистической регрессии: 0.026438569206842923
Accuracy лучшей модели логистической регресии: 0.7231726283048211


In [28]:
dummy_predictions = [0 for i, _ in enumerate(target_test)]
print(accuracy_score(target_test, dummy_predictions))

0.7122861586314152


**Вывод** 

По полученным значениям можно сказать, что все модели могут считаться адекватными, поскольку accuracy финальных моделей выше accuracy "глупых" моделей.

## Общий вывод

В ходе работы мы:
* разбили данные на 3 выборки: обучающая, валидационная, тестовая
* создали 3 модели: дерево решений, случайный лес, логистическая регрессия
* определили лучшие параметры и гиперпараметры
* определили лучшую модель
* оценили адекватность моделей

Модели обучались на выборке с размером в 1928 объектов. Лучшей моделью стало дерево решений с глубиной 6, критерием entropy и разветвителем best. Слачайный лес также показал хороший результат, а логистическая регрессия явный аутсайдер. Но можно смело сказать, что 2 модели показали хороший результат на тестовой выборке, ибо правильность этих моделей выросла. 

Самая высокая адекватность оказалась у случайного леса и дерева решений. По правильности и оценки адекватности можно сказать, что дерево решений и случайный лес отлично подойдут для подобной задачи.